In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
import numpy as np

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
full_X_train_transformed = pd.read_csv('CSV/pre-processed/full_X_train_transformed')
full_y_train = pd.read_csv('CSV/pre-processed/full_y_train')
full_y_train = full_y_train.drop(columns=['id'])
full_y_train = full_y_train.iloc[:, 0]



full_X_test_transformed = pd.read_csv('CSV/pre-processed/full_X_test_transformed')


In [4]:
# Extracting only the image data from the full_X_train_transformed DataFrame
image_data_train = full_X_train_transformed.iloc[:, -40000:]
image_data_test = full_X_test_transformed.iloc[:, -40000:]

In [5]:
print(image_data_train.shape)
print(image_data_test.shape)
print(full_y_train.shape)

(990, 40000)
(594, 40000)
(990,)


In [6]:
image_data_train.columns = range(40000)
image_data_test.columns = range(40000)

In [12]:
image_data_train = image_data_train.values.reshape(-1, 200, 200)
image_data_test = image_data_test.values.reshape(-1, 200, 200)

In [13]:
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(full_y_train)

# Splitting 20% of the training data as a validation set
X_train, X_val, y_train_encoded_split, y_val_encoded_split = train_test_split(
    image_data_train, y_train_encoded, test_size=0.2, stratify=y_train_encoded, random_state=42
)

print(f"Training set size: {X_train.shape}")
print(f"Validation set size: {X_val.shape}")


Training set size: (792, 200, 200)
Validation set size: (198, 200, 200)


In [18]:
# Convert grayscale images to 3-channel (RGB) format
X_train_rgb = np.stack((X_train,) * 3, axis=-1)
X_val_rgb = np.stack((X_val,) * 3, axis=-1)

# Convert the RGB numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train_rgb.transpose(0, 3, 1, 2), dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val_rgb.transpose(0, 3, 1, 2), dtype=torch.float32).to(device)

# Normalize using ImageNet statistics
mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1).to(device)
std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1).to(device)
X_train_normalized = (X_train_tensor - mean) / std
X_val_normalized = (X_val_tensor - mean) / std


In [19]:
from torchvision.models.resnet import ResNet50_Weights

# Load pre-trained ResNet-50
model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Modify the final layer
num_classes = 10  # Change this to your number of classes
model.fc = nn.Linear(model.fc.in_features, num_classes)

In [20]:
from torchvision.models.resnet import ResNet50_Weights

# The image data tensors were already created as X_train_normalized and X_val_normalized
# Convert label data to tensors
y_train_tensor = torch.tensor(y_train_encoded_split, dtype=torch.int64).to(device)
y_val_tensor = torch.tensor(y_val_encoded_split, dtype=torch.int64).to(device)

# Define the model using ResNet50
n_classes = full_y_train.nunique()
model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, n_classes)  # Modify the final layer

# Optionally, freeze all layers
for param in model.parameters():
    param.requires_grad = False
# Unfreeze the final layer
for param in model.fc.parameters():
    param.requires_grad = True

model.to(device)
print(model)

# Define the loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [22]:
# Training parameters
epochs = 10
batch_size = 32

# Training loop
for epoch in range(epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        # Determine end index for the current batch
        end_idx = min(i + batch_size, len(X_train_tensor))
        
        # Get the mini-batch data
        inputs = X_train_tensor[i:end_idx]
        labels = y_train_tensor[i:end_idx]

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Print statistics
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {loss.item()}")

print('Finished Training')

Epoch [1/100] - Loss: 4.561861515045166
Epoch [2/100] - Loss: 4.273617267608643


KeyboardInterrupt: 

In [ ]:
# Set the model to evaluation mode
model.eval()

correct_predictions = 0
total_predictions = 0

# Prevent gradient calculations
with torch.no_grad():
    for i in range(0, len(X_val_tensor), batch_size):
        # Determine end index for the current batch
        end_idx = min(i + batch_size, len(X_val_tensor))
        
        # Get the mini-batch data
        inputs = X_val_tensor[i:end_idx]
        labels = y_val_tensor[i:end_idx]

        # Forward pass
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

accuracy = 100 * correct_predictions / total_predictions
print(f'Accuracy on the validation set: {accuracy:.2f}%')
